In [ ]:
import pandas as pd

# Importing the dataset
DATASET_COLUMNS=['target','ids','date','flag','user','text']
DATASET_ENCODING = "ISO-8859-1"
df = pd.read_csv('../input/sentiment140/training.1600000.processed.noemoticon.csv', encoding=DATASET_ENCODING, names=DATASET_COLUMNS)
df.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
data=df[['text','target']]
data.head()

,text,target
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
1,is upset that he can't update his Facebook by ...,0
2,@Kenichan I dived many times for the ball. Man...,0
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",0


In [ ]:
data['target'] = data['target'].replace(4,1)
data['target'].value_counts()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


1    800000
0    800000
Name: target, dtype: int64

In [ ]:
data_pos = data[data['target'] == 1]
data_pos.head()

,text,target
800000,I LOVE @Health4UandPets u guys r the best!!,1
800001,im meeting up with one of my besties tonight! ...,1
800002,"@DaRealSunisaKim Thanks for the Twitter add, S...",1
800003,Being sick can be really cheap when it hurts t...,1
800004,@LovesBrooklyn2 he has that effect on everyone,1


In [ ]:
data_neg = data[data['target'] == 0]
data_neg.head()

,text,target
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
1,is upset that he can't update his Facebook by ...,0
2,@Kenichan I dived many times for the ball. Man...,0
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",0


In [ ]:
import numpy as np
data_pos = data_pos.sample(100000)
data_neg = data_neg.sample(100000)
dataset = pd.concat([data_pos, data_neg])
dataset = dataset.iloc[np.random.permutation(len(dataset))]
dataset=dataset.reset_index(drop=True)
dataset.head()

,text,target
0,Ordered our new washing machine to replace the...,0
1,@bsdgypsy You can't have me unless you're prep...,1
2,@GreerMcDonald good luck on the whole socket i...,1
3,@arjunghosh Ya i know. I must be one of her gr...,0
4,I gotta go already Tweet you all laterrrrrrrr...,0


In [ ]:
X=list(dataset['text'])
X[:5]

['Ordered our new washing machine to replace the broken one. Another ï¿½350 lighter now  it is an 8kg one though!',
 "@bsdgypsy You can't have me unless you're prepared to take down Kirstie... And her followers... I WISH YOU LUCK! :p ",
 '@GreerMcDonald good luck on the whole socket issue...  even in taiwan I struggled!',
 '@arjunghosh Ya i know. I must be one of her greatest fans, too bad she never followed me back ',
 'I gotta go already  Tweet you all laterrrrrrrrrrrr!']

In [ ]:
y=list(dataset['target'])
y[:5]

[0, 1, 1, 0, 0]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42,stratify=y)
X_train[:5]

['good morninggg, so tired ',
 'bored. wishing i was in philly for the roh tapings ',
 'ehhh. last day in math@bio. be home @  1. tomorrow, lastday at the annex everrrr ',
 "Holy Burritos! I don't want to go to church..  I want to sit at home and... get fat. lol.",
 '@loonyhiker what a night!  hope you get a nap today ']

In [ ]:
!pip install transformers

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [ ]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [ ]:
y_train[:5]

[0, 0, 1, 0, 1]

In [ ]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

In [ ]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.1,               # strength of weight decay
    logging_steps=100,
)

In [ ]:
with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = TFTrainer(
    model=model,                         # the instantiated 
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)

trainer.train()

In [ ]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.3736282470703125}

In [ ]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[-1.4072479 ,  1.5133042 ],
       [-2.5039785 ,  2.4736738 ],
       [ 2.0901103 , -1.9074324 ],
       ...,
       [-0.22562572,  0.18473253],
       [ 1.4977549 , -1.4138759 ],
       [ 1.7289667 , -1.5435936 ]], dtype=float32), label_ids=array([1, 1, 0, ..., 1, 0, 0], dtype=int32), metrics={'eval_loss': 0.3736282470703125})

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,trainer.predict(test_dataset)[1]))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     20000
           1       1.00      1.00      1.00     20000

    accuracy                           1.00     40000
   macro avg       1.00      1.00      1.00     40000
weighted avg       1.00      1.00      1.00     40000

